In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import collections
import string
from datetime import datetime as dt
from datetime import timedelta
import os

In [2]:
os.chdir('../')
path = os.getcwd()
print(path)

C:\home\program


In [3]:
NEWS = pd.read_csv('scraping/category2018-2019.csv')
FX = pd.read_csv('FX/円ドル(2014-1_2020-11).csv')
SPORT = pd.read_csv('scraping/sport2013-2020.csv')

In [4]:
NEWS.set_index('date')
dateTimeIndex = pd.DatetimeIndex(NEWS['date'])
NEWS.index = dateTimeIndex
NEWS = NEWS.drop("date", axis=1)

In [5]:
FX = FX.rename(columns={'日付': 'date'})
FX.set_index('date')
dateTimeIndex = pd.DatetimeIndex(FX['date'])
FX.index = dateTimeIndex
FX = FX.drop_duplicates()
FX = FX.resample('D').interpolate()
FX = FX.drop("date", axis=1)

In [6]:
FX['DIFF'] = FX.diff()
FX = FX['2018-01-01':'2019-01-31']
NEWS = NEWS['2018-01-01':'2019-01-31']

In [7]:
diff = FX['DIFF']

In [8]:
NEWS['DIIF-FX'] = diff

In [9]:
NEWS

,1,2,3,4,5,category,DIIF-FX
date,,,,,,,
2018-01-01,派,主義,##ト,##派,##ル,0,-0.01
2018-01-01,・,都,の,都立,、,2,-0.01
2018-01-01,、,に,中,から,の,2,-0.01
2018-01-01,日,年,##日,月曜日,分,2,-0.01
2018-01-01,差別,虐殺,迫害,飢餓,暴力,2,-0.01
...,...,...,...,...,...,...,...
2019-01-31,NaN,NaN,NaN,NaN,NaN,0,-0.15
2019-01-31,NaN,NaN,NaN,NaN,NaN,0,-0.15
2019-01-31,NaN,NaN,NaN,NaN,NaN,0,-0.15


In [10]:
NEWS['weight'] = 0
for i in range(0,len(NEWS)):
    if NEWS['DIIF-FX'][i] > 1:
        NEWS['weight'][i] = 1
    elif NEWS['DIIF-FX'][i] < -1:
        NEWS['weight'][i] = -1
    else:
        NEWS['weight'][i] = 0

C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
Counter(NEWS['weight'])

Counter({0: 80563, -1: 1918, 1: 849})

# sport

In [15]:
SPORT.set_index('Unnamed: 0')
dateTimeIndex = pd.DatetimeIndex(SPORT['Unnamed: 0'])
SPORT.index = dateTimeIndex
sport = SPORT.drop('Unnamed: 0', axis=1)

In [16]:
sport['DIFF'] = sport.diff()
NEWS['sport'] = sport['DIFF']

In [17]:
NEWS['sport']

date
2018-01-01   -0.645161
2018-01-01   -0.645161
2018-01-01   -0.645161
2018-01-01   -0.645161
2018-01-01   -0.645161
                ...   
2019-01-31   -0.225806
2019-01-31   -0.225806
2019-01-31   -0.225806
2019-01-31   -0.225806
2019-01-31   -0.225806
Name: sport, Length: 83330, dtype: float64

In [18]:
NEWS['weight-S'] = 0
for i in range(0,len(NEWS)):
    if NEWS['sport'][i] > 0.1:
        NEWS['weight-S'][i] = 1
    elif NEWS['sport'][i] < -0.1:
        NEWS['weight-S'][i] = -1
    else:
        NEWS['weight-S'][i] = 0

C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
Counter(NEWS['weight-S'])

Counter({-1: 32040, 1: 38671, 0: 12619})

In [20]:
NEWS['total'] = 0
for i in range(0,len(NEWS)):
    if NEWS['category'][i] == 1:
        NEWS['total'][i] += NEWS['weight'][i]
    elif NEWS['category'][i] == 2:
        NEWS['total'][i] += NEWS['weight-S'][i]

C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\home\anaconda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
Counter(NEWS['total'])

Counter({0: 48174, -1: 15865, 1: 19291})

In [22]:
dateTimeIndex = FX['2018-01-01':'2019-12-09'].index
Weight = pd.DataFrame(index=[dateTimeIndex],columns=['weight','FX'])
Weight['weight'] = 0
k=0
for i in range(0,len(NEWS)):
    try:
        if NEWS.index[i] == NEWS.index[i+1]:
            Weight.iat[k,0] += NEWS['total'][i]
        else:
            Weight.iat[k,0] += NEWS['total'][i]
            Weight.iat[k,1] = FX.iat[k,0]
            k+=1
    except IndexError:
        if (i+1) == len(NEWS):
            Weight.iat[k,0] += NEWS['total'][i]
            Weight.iat[k,1] = FX.iat[k,0]

In [23]:
Counter(Weight['weight'])

Counter({-22: 2,
         21: 5,
         67: 1,
         72: 2,
         176: 3,
         101: 2,
         28: 3,
         35: 1,
         68: 1,
         153: 4,
         235: 1,
         254: 1,
         88: 1,
         59: 1,
         53: 2,
         222: 1,
         180: 1,
         210: 1,
         204: 1,
         86: 2,
         55: 2,
         45: 3,
         194: 1,
         132: 1,
         172: 2,
         182: 2,
         58: 1,
         26: 3,
         23: 1,
         171: 1,
         173: 3,
         151: 1,
         -163: 1,
         -42: 6,
         -16: 4,
         -7: 1,
         -149: 1,
         -161: 3,
         -154: 2,
         -171: 1,
         -27: 1,
         -9: 2,
         -33: 1,
         -144: 3,
         -147: 1,
         -148: 4,
         -28: 1,
         -11: 1,
         -13: 2,
         -123: 1,
         -142: 1,
         -47: 1,
         -8: 1,
         -6: 1,
         -158: 2,
         -136: 2,
         0: 61,
         -166: 2,
         -93: 1,
    

In [24]:
Weight.to_csv("scraping/Weight-FXandSPORT2018-2019.csv")